In [2]:
import random # Choosing the random response
import json
import pickle # for serialization
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer # to reduce to stem word
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD, Adam
import torch
import random
from tensorflow.keras.models import load_model
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer, util
sentence_model = SentenceTransformer('all-MiniLM-L6-v2')
import nltk
nltk.download('punkt')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79 kB 4.9 MB/s 
     |████████████████████████████████| 4.2 MB 58.0 MB/s 
     |████████████████████████████████| 1.2 MB 48.2 MB/s 
     |████████████████████████████████| 86 kB 5.8 MB/s 
     |████████████████████████████████| 6.6 MB 43.3 MB/s 
     |████████████████████████████████| 596 kB 72.4 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=8b1f0973456dba07c1c233d9a3c31a43033d64fbef391a63e4f768a7d1d06f6e
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
!pip install nltk


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [29]:
with (open('data.json')) as f:
    intents = json.load(f)
with (open('material_rate.json')) as f:
    material_rates = json.load(f)

documents = []
classes = []
words = []

ignore_letters = ['?', '!', '.', ',']


# New Section

In [6]:
from tensorflow.keras.models import Sequential
corpus = []
for intent in intents['intents']:
  for pattern in intent['patterns']:
    # print(pattern, intent['tag'])
    pattern = pattern.lower()
    corpus.append(pattern)
    pattern_embedding = model.encode(pattern, convert_to_tensor = True)
    # print(len(pattern_embedding.tolist()))
    # print(pattern_embedding.shape)
    documents.append((pattern_embedding.tolist(), intent['tag']))  # documents conatins training data -> (phrase, class) - (["How","are","you"], "greetings")
    if intent['tag'] not in classes:
      classes.append(intent['tag'])
print('classes =',classes)
print(pattern_embedding.shape)      # Features in the embedding - 384


classes = ['greetings', 'goodbye', 'Jewellery', 'name', 'negative feedback', 'positive feedback', 'Specialization', 'Random', 'materialpricegivenweight', 'discounts', 'demands/expectations', 'refund', 'deliverylocation', 'replaceitem', 'deliveryduration', 'timings', 'complaints']
torch.Size([384])


In [7]:
classes = sorted(set(classes))
print(len(classes))
training = []
#initialize the model output with zeros - for one hot encoding
output_empty = [0] * len(classes)

import pickle
classes_file='classes.pkl'
f = open(classes_file,'wb')
pickle.dump(classes,f)
f.close()

17


# New Section

In [8]:
# training = []
data = []
target = []

for document in documents:    #embedding
    output_row = list(output_empty)
    #setting the label for the class
    output_row[classes.index(document[1])] = 1
    data.append(document[0])
    target.append(output_row)

train_x = data
print(train_x)
train_y = target
print(np.array(data).shape)
print(np.array(target).shape)




[[-0.09047617763280869, 0.04043964296579361, 0.023905690759420395, 0.05894798785448074, -0.022882327437400818, -0.047220125794410706, 0.045047614723443985, 0.01578637585043907, -0.048199500888586044, -0.03779415786266327, -0.019077690318226814, 0.021308772265911102, -0.0046830023638904095, -0.04330819472670555, 0.05991475284099579, 0.059103384613990784, -0.02803676761686802, -0.05921836569905281, -0.12440314888954163, -0.03559993952512741, -0.0060805100947618484, 0.0324290506541729, -0.03780081495642662, 0.024710949510335922, -0.042724378407001495, -0.0424538291990757, 0.045935675501823425, 0.0986255407333374, -0.04999801516532898, -0.035235822200775146, 0.07083970308303833, 0.03316321596503258, 0.02658839523792267, 0.00017320108599960804, 0.003881648648530245, 0.0304672010242939, -0.07820260524749756, -0.1203794851899147, 0.018041526898741722, 0.022829048335552216, -0.0017750978004187346, -0.02344992198050022, 0.003058190457522869, 0.024355726316571236, 0.04415396600961685, -0.0401097

In [34]:
# cosine similarity
from sentence_transformers.util import cos_sim
def find_similarity(inputs,sentences):
    input_embeddings = sentence_model.encode([inputs], convert_to_tensor = True)
    corpus_embeddings = sentences
    cosine_scores = []
    cosine_scores = cos_sim(corpus_embeddings, input_embeddings)
    #Find the pair with the highest cosine similarity score
    pair = []
    max_score = -1
    max_index = -1
    corpus_score_dict = dict(zip(corpus, cosine_scores))
    # print('corpus_score_dict = ',corpus_score_dict)
    for i in range(len(cosine_scores)):
        if cosine_scores[i]>max_score:
            max_score = cosine_scores[i]
            max_index = i
    found_tag = -1
    tag = ""
    responses = []
    pair = [inputs, corpus[max_index]]
    for intent in intents['intents']:
      tag = intent['tag']
      # print('tag = ', tag)
      patterns = intent['patterns']
      # print('patterns = ', patterns)
      for i in range(len(patterns)):
        # print('corpus[max_index],pattern[i] = ',corpus[max_index],' ', patterns[i])
        if corpus[max_index] == patterns[i].lower():
          responses = intent['responses']
          # print(responses)
          # print('Related tag = ', tag)
          found_tag = 1
          break
      if found_tag == 1:
        break
    # print('most matching sentence: \n', pair)
    # print('Similarity:', max_score)
    index = random.randint(0,len(responses)-1)
    print()
    return tag,responses[index]


In [32]:
def choosen_answers(checks_list, checks_json):
    selectult=""
    tag = checks_list[0]['intent']
    list_of_checks = checks_json['intents']
    for i in list_of_checks:
        if i['tag'] == tag:
            selectult = random.choice(i['responses'])
            break
    
    return selectult

def call_follow(class_is):
    num=random.randint(0,1)
    if num==1:
        for intent in checks['intents']:
            if intent['tag']==class_is:
                item = random.choice(intent['followup'])
                call_audio(item)


def call_audio(select):
    print(select)
    text_speech.say(select)
    text_speech.runAndWait()
    
def call_text():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Speak Customer:")
        audio = r.listen(source)
        try:
            text = r.recognize_google(audio)
            return text
        except:
            print("Sorry could not recognize what you said")
def get_txt_response(intents_list, intents_json):
  tag = intents_list[0]['intent']
  list_of_intents = intents_json['intents']
  for i in list_of_intents:
    if i['tag'] == tag:
      result = random.choice(i['responses'])
      break
  return result
  
def choose_follow(class_is):
    num=random.randint(0,1)
    if num==1:
      for intent in intents['intents']:
        tag = intent['tag']
        if tag == class_is:
          follow_ups = intent['followup']
          index = random.randint(0,len(follow_ups)-1)
          follow_up = intent['followup'][index]
          return follow_up


def get_price_response(material):
    response = ""
    materials = material_rates["material_rate"]
    count = 0
    found = 0
    # call_audio('We sell gold, silver, platinum and diamond. Which material would you like to know about first?')
    # material = call_text()
    # material = input("We sell gold, silver, platinum and diamond. Which material would you like to know about first?")
    while 1:
        if material.find('diamond')!=-1:
            response = 'The current rate for one 4 carat diamond is 329230 rupees per gram, 1/2 carat diamond is 65847 rupees per gram, and 1 carat is 131693 per gram.'
            found =1
        elif material.find("gold")!=-1:
            
            material = "gold"
            response = 'The current rate for 24 carat gold is 140500 rupees per tola, 22 carat gold is 128791 rupees per tola, 21 carat gold is 122938 per tola, and 18 carat gold is 105375 rupees per tola'
            found =1

        elif material.find("silver")!=-1:
            material = "silver"
            response = 'The current rate for silver is 1678 rupees per tola, and 144 rupees per gram'
            found =1

        elif material.find("platinum")!=-1:
            material = "platinum"
            response = 'The current rate for platinum is 72656 rupees per tola, and 6236 rupees per gram'
            found =1
        else:
            found = 0   # no relevant material present
            # call_audio('We sell gold, silver, platinum and diamond. Which material would you like to know about first?')
            # material = call_text()
            material = input("We sell gold, silver, platinum and diamond. Which material would you like to know about first?")
        if found == 1 or material.find("gold")!=-1 or material.find("silver")!=-1 or material.find("platinum")!=-1 or material.find("diamond")!=-1:
            break 
    return response

In [35]:
# test

# import speech_recognition as sr

print("Intelligent AI Chatbot. . . ")

while True:
    # message = call_text()
    message = input("")
    tag,response = find_similarity(message, train_x)
    # print(tag)
    if(tag=='goodbye'):
        # call_audio(response)
        print(response)
        break
        
    elif(tag =='materialpricegivenweight'):
        response=get_price_response(message)
        # call_audio(response)
        print(response)
        follow = choose_follow(tag)
        if follow is not None:
          print(follow)
    else:
        # call_audio(response)
        print(response)
        # call_follow(class_is)
        follow = choose_follow(tag)
        if follow is not None:
          print(follow)
   

Intelligent AI Chatbot. . . 
Hello !!

Welcome to our shop...May i help you
Please show me your best selling items

We have our specialization in jewellery making, mostly rings and bracelets.
Will you like to take a tour of our shop
Yes please, show me rings

Anything for our customers :) 
What's the rate for gold today?

The current rate for 24 carat gold is 140500 rupees per tola, 22 carat gold is 128791 rupees per tola, 21 carat gold is 122938 per tola, and 18 carat gold is 105375 rupees per tola
Ooh expensive!

We sell gold, silver, platinum and diamond. Which material would you like to know about first?silver

That was weird

Our team is always try to get the best for you!!!
In the mean time you can fill our feedback form.
Alright. Your grammar sucks

Our shop is all about making jewellery, even customized as well
Will you like to take a tour of our shop
Can you tell me your shop timings

Our shop is open from 8:00 am to 12:30 am
We have the longest hours in the market!
Those are 

In [ ]:
# print(len(training[0][0]))

# #trainingdata - embeddings

# # random.shuffle(training)
# print(len(training[0][0]))
# training = np.array(training)
# print(training.shape)
# print(training[:, 0].shape)

# train_x = list(training[:, 0])    # embeddings
# train_y = list(training[:, 1])    # the corresponding classes
# train_x_list = []
# train_y_list = []
# print(len(train_x[0]))


# for x in train_x:
#   train_x_list.append(x.tolist())

# for i in range(len(train_y)):
#   print(train_y[i])

# # print(train_x_list)
# # print(train_y_list)
# train_x = train_x_list
# # train_y = train_y_list

train_x = data
train_y = target
print(np.array(data).shape)
print(np.array(target).shape)

model = Sequential()
model.add(Dense(256, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))                 #dropout for regularizing - train 50% parameters at a single time
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

(106, 384)
(106, 17)


In [ ]:
#changed epochs, changed mini-batch size
# your_torch_tensor.numpy()

ai_model = model.fit(train_x, train_y, epochs=500, batch_size=30, verbose=1)
model.save('chatbot_embedding_model.h5', ai_model)
print("AI Chatbot Model Built")

Epoch 1/500
4/4 [==============================] - 0s 5ms/step - loss: 1.1077 - accuracy: 0.5755
Epoch 2/500
4/4 [==============================] - 0s 4ms/step - loss: 1.1870 - accuracy: 0.6132
Epoch 3/500
4/4 [==============================] - 0s 5ms/step - loss: 1.1203 - accuracy: 0.6132
Epoch 4/500
4/4 [==============================] - 0s 5ms/step - loss: 1.0014 - accuracy: 0.6604
Epoch 5/500
4/4 [==============================] - 0s 5ms/step - loss: 1.1431 - accuracy: 0.6038
Epoch 6/500
4/4 [==============================] - 0s 5ms/step - loss: 1.1271 - accuracy: 0.5943
Epoch 7/500
4/4 [==============================] - 0s 5ms/step - loss: 1.1531 - accuracy: 0.5566
Epoch 8/500
4/4 [==============================] - 0s 5ms/step - loss: 1.0970 - accuracy: 0.6038
Epoch 9/500
4/4 [==============================] - 0s 5ms/step - loss: 1.2328 - accuracy: 0.6226
Epoch 10/500
4/4 [==============================] - 0s 5ms/step - loss: 1.1421 - accuracy: 0.5849
Epoch 11/500
4/4 [===========